# Automatyczna klasyfikacja terenu z wykorzystaniem uczenia maszynowego

Zrodlo danych: Sentinel T34UDE_20200815T095039
Kroki wykonane przed analizą:
* Zmiana formatu warstw z jp2 na png ze względu na bezproblemową współpracę z OpenCV
* Klasyfikacja terenu z QGIS oraz wtyczkę QuickOSM
 ** water
 ** forest
 ** farmland
* Dla każdego typu terenu utworzono maskę w formie obrazu PNG o rozdzielczości zgodnej z danymi wejściowymi
* Utworzenie pliku konfiguracyjnego config.ini

#### Wczytanie bibliotek

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from configparser import ConfigParser
from time import time

In [ ]:
import preprocesing as pre
import helpers as hlp

#### Wczytanie danyc z pliku konfiguracyjnego

In [ ]:
config = ConfigParser()
config.read('config.ini')
input_dir = config['main']['input_dir']# Folder ze zdjęciami z Sentinela
classes_file = config['main']['classification_data']# Folder z maskami klas

Zdjęcia o rozdzielczości 10m składają sie z ponad 100 milionów pikseli
zatem do analizy wykorzystam tylko jego fragment o rozmiarze dx na dy
i zaczynający się od piksela (x_star, y_start)

In [ ]:
dx = int(config['main']['x_size'])
dy = int(config['main']['y_size'])
x_start = int(config['main']['x_start'])
y_start = int(config['main']['y_start'])
csv_data_file = config['main']['csv_data_file']

#### Przekształcamy dane wejściowe w coś przyjemniejszego do analizy

In [ ]:
data, columns_names = pre.images_to_numpy(input_dir, dx, dy, x_start, y_start)

In [ ]:
hlp.plot_MinMaxAvg(data, columns_names)

In [ ]:
hlp.plot_values_histogram(data, columns_names, ncols=4)

Rozkład wartości pikseli wskazuje na występowanie wartości odstających zatem przekształćmy je w następujący sposób: $ x = min(x,\overline{x}+3\sigma_{x}) $ oraz przeskalujmy z wykorzystaniem minmaxscaler z sklearn.

In [ ]:
data = pre.remove_outstandings(data)

In [ ]:
hlp.plot_MinMaxAvg(data, columns_names)

In [ ]:
hlp.plot_values_histogram(data, columns_names, ncols=4)

#### Wczytajmy teraz maski klas oraz stwórzmy klasę "other"

In [ ]:
classes, classes_names = pre.get_classes(classes_file, dx, dy, x_start, y_start)
other = (1 - classes.any(axis=1).astype(int)).reshape(-1, 1)
classes_names += ['other']
pre.add_classes_to_config(config, classes_names)
columns_names += classes_names 
nr_of_classes = len(classes_names)

In [ ]:
hlp.show_classes_distribution(np.concatenate((classes, other), axis=1), classes_names)

#### Łączymy wszystko

In [ ]:
data = np.concatenate((data, classes, other), axis=1)
data = pd.DataFrame(data, columns=columns_names)
data[classes_names] = data[classes_names].astype('int')

In [ ]:
data.head()

#### Dane zostały przygotowane zapisujemy je i możemy zająć się klasyfikacją

In [ ]:
data.to_csv(csv_data_file)

#### Dzielimy dane

In [ ]:
X = data.iloc[:,1:-nr_of_classes].to_numpy()
Y = data.iloc[:,-nr_of_classes:].to_numpy()
xyz = (dx,dy,nr_of_classes)

Zwalniamy trochę pamięci

In [ ]:
del data
del classes
del other

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

# Klasyfikacja obszaru z wykorzystaniem lasów losowych

#### Trenowanie

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
tic = time()
clf.fit(X_train, Y_train)
time_RF = time() - tic

#### Testowanie

In [ ]:
Y_pred_RF = clf.predict(X_test)

In [ ]:
print("Random forest acc: ",accuracy_score(Y_test, Y_pred_RF))

#### Wizualizacja wyników

In [ ]:
Y_pred_RF = clf.predict(X)
Y_pred_RF = np.rint(Y_pred_RF)
hlp.show_target_pred_dif(Y.reshape(xyz), Y_pred_RF.reshape((dx, dy, nr_of_classes)))

# Klasyfikacja obszaru z wykorzystaniem sieci neuronowych

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

#### Przygotowujemy model

In [ ]:
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax')
])

In [ ]:
#### TODO dodanie warstwy conwoluacyjnej powinno poprawić skuteczność. 

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Trenujemy

In [ ]:
tic = time()
history = model.fit(X_train, Y_train, validation_data = (X_test, Y_test), shuffle = 'True', epochs=50, batch_size=100000)

In [ ]:
time_DL = time()-tic

In [ ]:
_, accuracy = model.evaluate(X_test, Y_test)
print(f'Deap learning acc: {accuracy}')

In [ ]:
hlp.plot_keras_history(history)

Dokładność na danych testowych jest wyższa niż na danych uczących ze względu na niedokładności ręcznej klasyfikacji

#### Wizualizacja wyników

In [ ]:
Y_pred_DL = model.predict(X)
Y_pred_DL = np.rint(Y_pred_DL)
hlp.show_target_pred_dif(Y.reshape(xyz), Y_pred_RF.reshape((dx, dy, nr_of_classes)))

# Klasyfikacja obszaru z wykorzystaniem samoorganizujących się map

In [ ]:
from minisom import MiniSom
x_som, y_som = 4,4
som = MiniSom(x=x_som, y=y_som, input_len=X.shape[1], sigma=1.0, learning_rate=0.5)
som.random_weights_init(X)
som.train_random(X, num_iteration=100000, verbose=False)

#### Klasyfikujemy
Każdemu punktowi możemy przypisać jeden z neuronów mapy

In [ ]:
Y_pred_SOM = [som.winner(x) for x in X]
Y_pred_SOM = np.array([str(i) for i in Y_pred_SOM ])

#### One Hot Encode

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
Y_pred_SOM = enc.fit_transform(Y_pred_SOM.reshape(-1, 1)).toarray()

In [ ]:
hlp.show_classes_distribution(Y_pred_SOM, list(range(Y_pred_SOM.shape[1])))

In [ ]:
mapa = np.sum(Y_pred_SOM*som.distance_map().flatten(), axis=1)

#### rysujemy mapę

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(10,4), ncols=2, nrows=1)
ax1.set_title('SOM')
ax1.imshow(som.distance_map())
for (i, j), z in np.ndenumerate(som.distance_map()):
    ax1.text(j, i, '{:0.2f}'.format(som.distance_map()[i,j]), ha='center', va='center',color = 'white')
ax2.set_title('After classification')
ax2.imshow(mapa.reshape((dx,dy)))

In [ ]:
import cv2
cv2.imwrite('wynik_som.png', mapa.reshape((dx,dy))*255)

#### Walidacja

In [ ]:
clusstered = np.zeros((x_som,y_som,3))
matrix_IoU = hlp.metrics_matrix(Y, Y_pred_SOM, hlp.IoU)
clusstered[...,2]=matrix_IoU[:,2].reshape((x_som,y_som))#Klasa 1 kolor niebieski
clusstered[...,1]=matrix_IoU[:,1].reshape((x_som,y_som))#Klasa 2 kolor zielony
clusstered[...,0]=matrix_IoU[:,0].reshape((x_som,y_som))#Klasa 3 kolor czerwony
fig, ax = plt.subplots()
ax.set_title('Intersection over union')
plt.imshow(clusstered)
for (i, j, k), z in np.ndenumerate(clusstered):
    if z > 0.05:
        ax.text(j, i, '{:0.2f}'.format(max(clusstered[i,j,:])), ha='center', va='center',color = 'white')

Kolorami oznaczono klasę którą reprezentują. Słabe wyniki spowodowane są dużo wyższą liczbą otrzymanych klas niż klas które mieliśmy początkowo.

In [ ]:
best = [matrix_IoU[:,i].argsort()[-1] for i in range(3)]

In [ ]:
hlp.show_target_pred_dif(Y.reshape(xyz)[...,:-1], Y_pred_SOM.reshape((dx,dy,Y_pred_SOM.shape[1]))[...,best])

In [ ]:
yt, yp = Y.reshape(xyz)[...,1],  Y_pred_RF.reshape(xyz)[...,1]
cv2.imwrite('RF.png',(yt-yp+1)*127)

In [ ]:
yt, yp = Y.reshape(xyz)[...,1],  Y_pred_DL.reshape(xyz)[...,1]
cv2.imwrite('DL.png',(yt-yp+1)*127)

# Walidacja:

Jako miarę poprawności algorytmu wybrałem 
Indeks Jaccarda w skrócie IoU
$$IoU = \frac{A \cap B}{A \cup B} $$

TODO dodać inne metryki

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

def acc(cm):
    accuracy = []
    for i in range(cm.shape[0]):
        accuracy.append((np.sum(cm) - np.sum(cm[i,:]) - np.sum(cm[:,i]) + 2*cm[i,i])/np.sum(cm))
    return accuracy

def prec(cm):
    precision = []
    for i in range(cm.shape[0]):
        precision.append(cm[i,i]/(np.sum(cm[:,i])))
    return precision

In [ ]:
IoU_DL = [hlp.IoU(Y[...,i],Y_pred_DL[...,i]) for i in range(4)]

In [ ]:
IoU_RF = [hlp.IoU(Y[...,i],Y_pred_RF[...,i]) for i in range(4)]

In [ ]:
cm_DL = confusion_matrix(Y.argmax(axis=1), Y_pred_DL.argmax(axis=1))
cm_RF = confusion_matrix(Y.argmax(axis=1), Y_pred_RF.argmax(axis=1))

In [ ]:
validation = pd.DataFrame(columns=['model'] + classes_names)

In [ ]:
validation.loc[0]=(['IoU_DL']+IoU_DL)
validation.loc[1]=(['IoU_RF']+IoU_RF)
validation.loc[2]=(['Acc_DL']+hlp.acc(cm_DL))
validation.loc[3]=(['Acc_RF']+hlp.acc(cm_RF))
validation.loc[4]=(['Prec_DL']+hlp.prec(cm_DL))
validation.loc[5]=(['Prec_RF']+hlp.prec(cm_RF))
validation

In [ ]:
models_names = ['DL','RF', 'SOM']
colors = ['black', 'white']
fig = plt.figure(figsize=(10,12))
gs = fig.add_gridspec(3,2)
axs = []
axs.append(fig.add_subplot(gs[0, 0]))
axs.append(fig.add_subplot(gs[1, 0]))
axs.append(fig.add_subplot(gs[:, 1]))
axs.append(fig.add_subplot(gs[2, 0]))
axs[-1].axis('off')

for k, y in enumerate([Y_pred_DL, Y_pred_RF, Y_pred_SOM]):
    cm = hlp.metrics_matrix(Y,y)
    axs[k].matshow(cm, cmap='Blues')
    axs[k].set_title(models_names[k])
    axs[k].set(xlabel='target', ylabel='predicted')
    for (i, j), z in np.ndenumerate(cm):
            axs[k].text(j, i, '{:0.3f}'.format(z), ha='center', va='center',color = colors[int(round(z))])

TODO
* przenieść skrypt do helpers
* poprawić rozmieszczenie wykresów

# Prezentacja wyników

TODO

# Wnioski

TODO